In [ ]:
#create jupyter section header for navigation



In [ ]:
#import torch


# import

In [ ]:
#export
#import
import json
import os
from pathlib import Path
from typing import Optional, Union
from sklearn.metrics import mutual_info_score

import numpy as np
import scanpy as sc
import pandas as pd
import torch
import short_utils

from anndata import AnnData
from torch.utils.data import DataLoader, SequentialSampler
from tqdm import tqdm


from scgpt import logger
from scgpt.data_collator import DataCollator
from scgpt.model import TransformerModel
from scgpt.tokenizer import GeneVocab
from scgpt.utils import load_pretrained

import pyperclip



PathLike = Union[str, os.PathLike]

# batch embed func

In [ ]:
#export
def get_batch_cell_embeddings(
    adata,
    cell_embedding_mode: str = "cls",
    model=None,
    vocab=None,
    max_length=1200,
    batch_size=64,
    model_configs=None,
    gene_ids=None,
    use_batch_labels=False,
) -> np.ndarray:
    """
    Get the cell embeddings for a batch of cells.

    Args:
        adata (AnnData): The AnnData object.
        cell_embedding_mode (str): The mode to get the cell embeddings. Defaults to "cls".
        model (TransformerModel, optional): The model. Defaults to None.
        vocab (GeneVocab, optional): The vocabulary. Defaults to None.
        max_length (int): The maximum length of the input sequence. Defaults to 1200.
        batch_size (int): The batch size for inference. Defaults to 64.
        model_configs (dict, optional): The model configurations. Defaults to None.
        gene_ids (np.ndarray, optional): The gene vocabulary ids. Defaults to None.
        use_batch_labels (bool): Whether to use batch labels. Defaults to False.

    Returns:
        np.ndarray: The cell embeddings.
    """

    count_matrix = adata.X
    count_matrix = (
        count_matrix if isinstance(count_matrix, np.ndarray) else count_matrix.A
    )
    print("loaded count matrix")

    # gene vocabulary ids
    if gene_ids is None:
        gene_ids = np.array(adata.var["id_in_vocab"])
        assert np.all(gene_ids >= 0)

    if use_batch_labels:
        batch_ids = np.array(adata.obs["batch_id"].tolist())

    class Dataset(torch.utils.data.Dataset):
        def __init__(self, count_matrix, gene_ids, batch_ids=None):
            self.count_matrix = count_matrix
            self.gene_ids = gene_ids
            self.batch_ids = batch_ids

        def __len__(self):
            return len(self.count_matrix)

        def __getitem__(self, idx):
            row = self.count_matrix[idx]
            nonzero_idx = np.nonzero(row)[0]
            values = row[nonzero_idx]
            genes = self.gene_ids[nonzero_idx]
            # append <cls> token at the beginning
            genes = np.insert(genes, 0, vocab["<cls>"])
            values = np.insert(values, 0, model_configs["pad_value"])
            genes = torch.from_numpy(genes).long()
            values = torch.from_numpy(values)
            output = {
                "id": idx,
                "genes": genes,
                "expressions": values,
            }
            if self.batch_ids is not None:
                output["batch_labels"] = self.batch_ids[idx]
            return output

    if cell_embedding_mode == "cls":
        dataset = Dataset(
            count_matrix, gene_ids, batch_ids if use_batch_labels else None
        )
        print("created dataset")
        collator = DataCollator(
            do_padding=True,
            pad_token_id=vocab[model_configs["pad_token"]],
            pad_value=model_configs["pad_value"],
            do_mlm=False,
            do_binning=True,
            max_length=max_length,
            sampling=True,
            keep_first_n_tokens=1,
        )
        print("created collator")

        data_loader = DataLoader(
            dataset,
            batch_size=batch_size,
            sampler=SequentialSampler(dataset),
            collate_fn=collator,
            drop_last=False,
            num_workers=min(len(os.sched_getaffinity(0)), batch_size),
            pin_memory=True,
        )
        print("created data loader")

        device = next(model.parameters()).device
        print("created device")
        cell_embeddings = np.zeros(
            (len(dataset), model_configs["embsize"]), dtype=np.float32
        )
        print("created intial cell embeddings")
        with torch.no_grad(), torch.cuda.amp.autocast(enabled=True):
            count = 0
            for data_dict in tqdm(data_loader, desc="Embedding cells"):
                input_gene_ids = data_dict["gene"].to(device)
                src_key_padding_mask = input_gene_ids.eq(
                    vocab[model_configs["pad_token"]]
                )
                print(" input gene ids to device")
                embeddings = model._encode(
                    input_gene_ids,
                    data_dict["expr"].to(device),
                    src_key_padding_mask=src_key_padding_mask,
                    batch_labels=data_dict["batch_labels"].to(device)
                    if use_batch_labels
                    else None,
                )
                print("encoded embeddings")

                embeddings = embeddings[:, 0, :]  # get the <cls> position embedding
                embeddings = embeddings.cpu().numpy()
                cell_embeddings[count : count + len(embeddings)] = embeddings
                count += len(embeddings)
        cell_embeddings = cell_embeddings / np.linalg.norm(
            cell_embeddings, axis=1, keepdims=True
        )
    else:
        raise ValueError(f"Unknown cell embedding mode: {cell_embedding_mode}")
    return cell_embeddings



# embedd all data func


In [ ]:
#export
def embed_data(
    adata_or_file: Union[AnnData, PathLike],
    model_dir: PathLike,
    cell_type_key: str = "cell_type",
    gene_col: str = "feature_name",
    max_length=1200,
    batch_size=64,
    obs_to_save: Optional[list] = None,
    device: Union[str, torch.device] = "cuda",
    use_fast_transformer: bool = True,
    return_new_adata: bool = False,
) -> AnnData:
    """
    Preprocess anndata and embed the data using the model.

    Args:
        adata_or_file (Union[AnnData, PathLike]): The AnnData object or the path to the
            AnnData object.
        model_dir (PathLike): The path to the model directory.
        cell_type_key (str): The key in adata.obs that contains the cell type labels.
            Defaults to "cell_type".
        gene_col (str): The column in adata.var that contains the gene names.
        max_length (int): The maximum length of the input sequence. Defaults to 1200.
        batch_size (int): The batch size for inference. Defaults to 64.
        obs_to_save (Optional[list]): The list of obs columns to save in the output adata.
            If None, will only keep the column of :attr:`cell_type_key`. Defaults to None.
        device (Union[str, torch.device]): The device to use. Defaults to "cuda".
        use_fast_transformer (bool): Whether to use flash-attn. Defaults to True.
        return_new_adata (bool): Whether to return a new AnnData object. If False, will
            add the cell embeddings to a new :attr:`adata.obsm` with key "X_scGPT".

    Returns:
        AnnData: The AnnData object with the cell embeddings.
    """
    if isinstance(adata_or_file, AnnData):
        adata = adata_or_file
    else:
        adata = sc.read_h5ad(adata_or_file)

    # verify cell type key and gene col
    assert cell_type_key in adata.obs
    if gene_col == "index":
        adata.var["index"] = adata.var.index
    else:
        assert gene_col in adata.var

    if device == "cuda":
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if not torch.cuda.is_available():
            print("WARNING: CUDA is not available. Using CPU instead.")

    # LOAD MODEL
    model_dir = Path(model_dir)
    vocab_file = model_dir / "vocab.json"
    model_config_file = model_dir / "args.json"
    model_file = model_dir / "best_model.pt"
    pad_token = "<pad>"
    special_tokens = [pad_token, "<cls>", "<eoc>"]

    # vocabulary
    vocab = GeneVocab.from_file(vocab_file)
    for s in special_tokens:
        if s not in vocab:
            vocab.append_token(s)
    adata.var["id_in_vocab"] = [
        vocab[gene] if gene in vocab else -1 for gene in adata.var[gene_col]
    ]
    gene_ids_in_vocab = np.array(adata.var["id_in_vocab"])
    logger.info(
        f"match {np.sum(gene_ids_in_vocab >= 0)}/{len(gene_ids_in_vocab)} genes "
        f"in vocabulary of size {len(vocab)}."
    )
    adata = adata[:, adata.var["id_in_vocab"] >= 0]

    with open(model_config_file, "r") as f:
        model_configs = json.load(f)

    # Binning will be applied after tokenization. A possible way to do is to use the unified way of binning in the data collator.

    vocab.set_default_index(vocab["<pad>"])
    genes = adata.var[gene_col].tolist()
    gene_ids = np.array(vocab(genes), dtype=int)

    # all_counts = adata.layers["counts"]
    # num_of_non_zero_genes = [
    #     np.count_nonzero(all_counts[i]) for i in range(all_counts.shape[0])
    # ]
    # max_length = min(max_length, np.max(num_of_non_zero_genes) + 1)

    model = TransformerModel(
        ntoken=len(vocab),
        d_model=model_configs["embsize"],
        nhead=model_configs["nheads"],
        d_hid=model_configs["d_hid"],
        nlayers=model_configs["nlayers"],
        nlayers_cls=model_configs["n_layers_cls"],
        n_cls=1,
        vocab=vocab,
        dropout=model_configs["dropout"],
        pad_token=model_configs["pad_token"],
        pad_value=model_configs["pad_value"],
        do_mvc=True,
        do_dab=False,
        use_batch_labels=False,
        domain_spec_batchnorm=False,
        explicit_zero_prob=False,
        use_fast_transformer=use_fast_transformer,
        fast_transformer_backend="flash",
        pre_norm=False,
    )
    load_pretrained(model, torch.load(model_file), verbose=False)
    model.to(device)
    model.eval()
    print("loaded model")

    # get cell embeddings
    cell_embeddings = get_batch_cell_embeddings(
        adata,
        cell_embedding_mode="cls",
        model=model,
        vocab=vocab,
        max_length=max_length,
        batch_size=batch_size,
        model_configs=model_configs,
        gene_ids=gene_ids,
        use_batch_labels=False,
    )
    print("got cell embeddings")

    if return_new_adata:
        obs_to_save = [cell_type_key] if obs_to_save is None else obs_to_save
        obs_df = adata.obs[obs_to_save]
        return sc.AnnData(X=cell_embeddings, obs=obs_df, dtype="float32")

    adata.obsm["X_scGPT"] = cell_embeddings
    return adata

# run embedding

In [ ]:
#check cuda available
torch.cuda.is_available()

In [ ]:
#print working directory:
print(os.getcwd())
#print working dir contents:
print(os.listdir(os.getcwd()))


In [ ]:
#set working directory:
#os.chdir("/workspace")

In [ ]:
#export
base_dir = short_utils.get_base_dir()
base_dir


In [ ]:
#export
# load data
full_adata = sc.read_h5ad(base_dir / 'training_data/tcga/genexp_data/xena_pan_can_genexp_clin.h5ad')
# load and examine the data at data/brca_scrna_epithelial.h5ad



In [ ]:
#export
#truncate the data to 1000 cells
my_adata = full_adata[:10]

In [ ]:
#deal with na
my_adata.obs['tumor_type'].isna().sum()
#var = my_adata.var
#var.isna().sum()

In [ ]:
#replace nan with 'unknwon'

col_name = 'tumor_type'

# Add 'unknown' to the categories
if 'unknown' not in my_adata.obs[col_name].cat.categories:
    my_adata.obs[col_name] = my_adata.obs[col_name].cat.add_categories('unknown')

# Now you can fill NaN values with 'unknown'
my_adata.obs[col_name] = my_adata.obs[col_name].fillna('unknown')


In [ ]:
#export
# prep args for embed:

#if plot by label, set the cell type arg to the cool with label

embed_args = {'adata_or_file': my_adata,
              'model_dir': Path(base_dir / 'scgpt/models/scGPT_pancancer'),
              'cell_type_key': "tumor_type",
                'gene_col': "hgnc_gene",
              'max_length' : 20000,
              'batch_size' : 1,
              'obs_to_save':  None,
              'device':  "cuda",
              'use_fast_transformer': False,
              'return_new_adata':  True,
              }

In [17]:
torch.cuda.empty_cache()

In [16]:
#export
cell_embbed = embed_data(**embed_args)

#clean cell output

scGPT - INFO - match 17350/20530 genes in vocabulary of size 60697.
loaded model
loaded count matrix
created dataset
created collator
created data loader
created device
created intial cell embeddings


Embedding cells:   0%|          | 0/10 [00:00<?, ?it/s]

 input gene ids to device
encoded embeddings


Embedding cells:  10%|█         | 1/10 [00:00<00:08,  1.01it/s]

 input gene ids to device
encoded embeddings


...

 input gene ids to device
encoded embeddings


Embedding cells:  90%|█████████ | 9/10 [00:08<00:00,  1.06it/s]

 input gene ids to device
encoded embeddings


Embedding cells: 100%|██████████| 10/10 [00:09<00:00,  1.08it/s]

got cell embeddings


consider to avoid memory outage.
figure out why reserving so much memory.
https://github.com/rentruewang/koila

In [ ]:
#export
#save the cell_embbed
cell_embbed.obs = my_adata.obs.copy()

In [ ]:
save_path = Path(base_dir / 'scgpt/data/bulk_brca_erbb2/tcga_all_scgpt_embb.h5ad')
cell_embbed.write_h5ad(save_path)

# add clinical data to adata

In [ ]:
#load clinical data 
clin_path = Path(base_dir / 'training_data/tcga/clinical/brca_tcga_pan_can_atlas_2018_clinical_data.tsv')

clin_df = pd.read_csv(clin_path, sep='\t', index_col=0)

In [ ]:
clin_df.iloc[0:5,:]

In [ ]:
cols = pd.Series(clin_df.columns)
keep_cols = [0,1,2,3,5,6,19,21,28,29,30,33,34,36,38,40,41,46,49,50,51,56]
#keep selected cols
clin_df = clin_df.iloc[:,keep_cols]

In [ ]:
clin_df = clin_df.set_index(['Sample ID'])
clin_df.index = clin_df.index.str.replace('-','.')


In [ ]:
#keep only samples in my_adata.obs['Sample_ID']
clin_df = clin_df.loc[my_adata.obs['Sample_ID'],:]

In [ ]:
#add clinical data to adata by joining on sample id
new_obs = pd.merge(my_adata.obs.copy(), clin_df, left_on='Sample_ID', right_on='Sample ID', how='left')

In [ ]:
#for each col, count na:
for col in new_obs.columns:
    print(col, new_obs[col].isna().sum())

In [ ]:

new_obs = new_obs.drop(['Neoplasm Histologic Grade'], axis=1) if 'Neoplasm Histologic Grade' in new_obs.columns else new_obs

In [ ]:
#add the new obs to cell em
cell_embbed.obs = new_obs

In [ ]:
cell_embbed.write_h5ad(base_dir / 'scgpt/data/bulk_brca_erbb2/tcga_brca_erbb2_scgpt_emb_oncosig_sub_genes_clin.h5ad')

In [ ]:
#add new obs to my adata and make smaple id the index
my_adata.obs = new_obs
my_adata.obs.set_index('Sample_ID', inplace=True) 
#full_adata = sc.read_h5ad(base_dir / 'scgpt/data/bulk_brca_erbb2/tcga_brca_erbb2_oncosig_sub_genes.h5ad')
# save my_adata as 'tcga_brca_erbb2_oncosig_sub_genes_clin.h5ad'


In [ ]:
my_adata.write_h5ad(base_dir / 'scgpt/data/bulk_brca_erbb2/tcga_brca_erbb2_oncosig_sub_genes_clin.h5ad')

# plotting

In [ ]:
#export
#plot the result cell_embbed.X which is #num cell rows of 512 collums
import plotly.express as px

import umap
import matplotlib.pyplot as plt

In [ ]:
full_adata.X.shape

In [ ]:
#export
projection_data = my_adata.X

In [ ]:
#export
#fit the projection
reducer = umap.UMAP()
embedding = reducer.fit_transform(projection_data)

In [ ]:
#export
# create a PCA

from sklearn.decomposition import PCA
import pandas as pd

# Perform PCA on the embeddings
pca = PCA(n_components=2)
pca_result = pca.fit_transform(projection_data)

# Create a DataFrame for the PCA results
pca_df = pd.DataFrame(pca_result, columns=['PCA 1', 'PCA 2'])
pca_df.index = my_adata.obs.index

# If you have the obs data as a DataFrame named my_adata.obs, concatenate it with the PCA results
full_df = pd.concat([my_adata.obs, pca_df], axis=1)


In [ ]:
#export

# Create a PCA plot
fig = px.scatter(pca_df, x='PCA 1', y='PCA 2',color=my_adata.obs['tumor_type'], title="PCA Plot")
fig.update_layout(xaxis_title="PCA 1", yaxis_title="PCA 2")
fig.show()

In [ ]:
#export
#print explained variance ratio
print('explained variance by pc 1 & 2: ',pca.explained_variance_)

In [ ]:
#get the loading of all cols in

In [ ]:

# Assuming my_adata.obs is a DataFrame containing the original observational data
# Calculate correlations
correlation_matrix = full_df.corr()

# Display the correlation matrix
correlation_matrix


# select and analyze subsets based on umap

In [ ]:
#export
# Prepare your data
umap_x = embedding[:, 0]
umap_y = embedding[:, 1]
# Create a DataFrame for Plotly: add the UMAP cols to my_adata.obs
umap_df = pd.DataFrame()
umap_df['UMAP 1'] = umap_x
umap_df['UMAP 2'] = umap_y
umap_df.index = my_adata.obs.index
#add the obs data to the umap df
umap_df = pd.concat([my_adata.obs, umap_df], axis=1)

plt_title = 'UMAP all tcga scgpt all genes'


plotyly themes:   ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']
https://plotly.com/python/templates/
https://plotly.com/python/reference/layout/

In [ ]:
#export
import plotly.io as pio
# Create a Plotly figure
fig = px.scatter(umap_df, x='UMAP 1', y='UMAP 2', color='tumor_type',
                 color_discrete_sequence=px.colors.qualitative.Plotly,
                 title=plt_title,
                 labels={'Label': 'cancer'},
                 opacity=0.8)

# Update layout for background and size nand use seaborn theme
fig.update_layout(
    template='seaborn',
    width=1200,
    height=1000,
    legend_title_text='Cancer',
    xaxis_title='UMAP 1',
    yaxis_title='UMAP 2'
)


# Adjust marker size and opacity
fig.update_traces(marker=dict(size=5, opacity=0.8))


# Save the plot as an image file
#pio.write_image(fig, 'umap_plot.png')

# Show the plot
fig.show()

In [ ]:
print(umap_df.columns)
umap_df.head()

In [ ]:
#export
#save the plot
plots_dir = Path(base_dir / 'scgpt/plots')
fig.write_image(str(plots_dir / f'{plt_title}.png'))


In [ ]:
#get working dir
os.getcwd()

# analyze results

measure umap dffrentiation of groups

In [ ]:
#export
#cluster the umap
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score, fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score, homogeneity_completeness_v_measure, mutual_info_score
from sklearn.metrics.cluster import contingency_matrix

from scipy.stats import entropy







In [ ]:
#clustering_vals = umap_df[['UMAP 1', 'UMAP 2']]
clustering_vals = my_adata.X

In [ ]:
# dbscan
clustering = DBSCAN(eps=0.5, min_samples=5).fit(clustering_vals)
clusters = clustering.labels_

In [ ]:
#try other clustering methods
import hdbscan

# Fit the HDBSCAN model
clusterer = hdbscan.HDBSCAN(min_cluster_size=50, min_samples=10)
clusters = clusterer.fit_predict(clustering_vals)

# Adjust parameters like `min_cluster_size` and `min_samples` based on your dat

#print number of clusters
print('number of clusters: ', len(set(clusters)))

In [ ]:
from sklearn.mixture import GaussianMixture

# Fit the GMM model
gmm = GaussianMixture(n_components=21, covariance_type='full')
clusters = gmm.fit_predict(clustering_vals)

# Adjust `n_components` (number of clusters) and `covariance_type` based on data

In [ ]:
from sklearn.cluster import KMeans

# Fit the K-Means model
kmeans = KMeans(n_clusters=21, init='k-means++')
clusters = kmeans.fit_predict(clustering_vals)

In [ ]:
umap_df['cluster'] = clusters

In [ ]:
#export
# Create a Plotly figure
fig = px.scatter(umap_df, x='UMAP 1', y='UMAP 2', color='cluster',
                 color_discrete_sequence=px.colors.qualitative.Plotly,
                 title=plt_title,
                 labels={'Label': 'cancer'},
                 opacity=0.8)

# Update layout for background and size nand use seaborn theme
fig.update_layout(
    template='seaborn',
    width=600,
    height=500,
    legend_title_text='Cancer',
    xaxis_title='UMAP 1',
    yaxis_title='UMAP 2'
)


# Adjust marker size and opacity
fig.update_traces(marker=dict(size=5, opacity=0.8))

# Show the plot
fig.show()

In [ ]:
subtype_counts = umap_df.groupby(['cluster', 'tumor_type']).size().unstack(fill_value=0)


In [ ]:
#export
#subtype metric
#measure accuracy by checking if subtypes tended to be in homogeneous clusters

# Initialize accuracy dictionary with tumor types as keys
accuracy_dict = {tumor_type: 0 for tumor_type in umap_df['tumor_type'].unique()}

# Initialize counts dictionary with tumor types as keys and their counts as values
counts_dict = umap_df['tumor_type'].value_counts().to_dict()

# Iterate over each cluster
for cluster in umap_df['cluster'].unique():
    # Get the subset of the DataFrame that corresponds to the current cluster
    cluster_data = umap_df[umap_df['cluster'] == cluster]

    # Count the number of occurrences of each tumor type in this cluster
    type_counts_in_cluster = cluster_data['tumor_type'].value_counts()

    # Calculate the clustering accuracy for each tumor type
    for tumor_type, count_in_cluster in type_counts_in_cluster.items():
        # Proportion of samples in the cluster of this subtype
        accuracy_in_cluster = count_in_cluster / cluster_data.shape[0]

        # Proportion of total samples of this subtype in the cluster
        prop_of_subtype_in_cluster = count_in_cluster / counts_dict[tumor_type]

        # Multiply and add to the accuracy dictionary for the tumor type
        accuracy_dict[tumor_type] += accuracy_in_cluster * prop_of_subtype_in_cluster
#sort the dict by values desc
#accuracy_dict = dict(sorted(accuracy_dict.items(), key=lambda item: item[1], reverse=True))

# create a df of tumor type and accuracy
accuracy_df = pd.DataFrame.from_dict(accuracy_dict, orient='index', columns=['accuracy'])



# Calculate the weighted average isolation accuracy
weighted_avg_isolation = sum(accuracy_dict[tumor_type] * counts_dict[tumor_type] for tumor_type in accuracy_dict) / (umap_df.shape[0])


# Print the weighted average isolation accuracy
print(f'Weighted average isolation accuracy: {weighted_avg_isolation:.4f}')
#print the df
accuracy_df

In [ ]:
#export
#cluster metric
#measure entropy of each cluster
entropy_scores = {}

# Initialize the sum of weighted entropies and total number of samples
weighted_entropy_sum = 0
total_samples = umap_df.shape[0]

# Iterate over all unique clusters and measure the entropy of the tumor_type distribution
for cluster in umap_df['cluster'].unique():
    # Get the tumor_type distribution of the cluster
    cluster_distribution = subtype_counts.loc[cluster]

    # Calculate the entropy for the cluster's tumor_type distribution
    # Convert the counts to a probability distribution (summing to 1)
    cluster_prob_dist = cluster_distribution / cluster_distribution.sum()
    cluster_entropy = entropy(cluster_prob_dist)

    # Store the entropy score for the cluster
    entropy_scores[cluster] = cluster_entropy

    #measure total entropy
    cluster_size = umap_df[umap_df['cluster'] == cluster].shape[0]
    weighted_entropy_sum += entropy_scores[cluster] * cluster_size


# Print the entropy scores
#for cluster, e_score in entropy_scores.items():
#    print(f'Entropy for cluster {cluster}: {e_score:.4f}')


# Calculate the weighted average entropy across all clusters
weighted_avg_entropy = weighted_entropy_sum / total_samples

# Print the weighted average entropy
#print(f'Weighted average entropy across all clusters: {weighted_avg_entropy:.4f}')


# Calculate the mutual information between the cluster labels for the entire dataset and the true labels
mi_score = mutual_info_score(umap_df['tumor_type'], umap_df['cluster'])
#print(f'Mutual Information for the entire clustering: {mi_score}')

#create a table with the avg entorpy and mi score where the metrics are the rows
metrics_df = pd.DataFrame({'avg_iso_w' : [weighted_avg_isolation],'weighted_avg_entropy': [weighted_avg_entropy], 'mi_score': [mi_score]}).T
metrics_df

In [ ]:
from sklearn.metrics import silhouette_score, fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score, homogeneity_completeness_v_measure

# Assuming umap_df contains your data, with 'tumor_type' as true labels and 'cluster' as predicted labels
true_labels = umap_df['tumor_type']
predicted_labels = umap_df['cluster']

# Silhouette Score
silhouette = silhouette_score(clustering_vals, predicted_labels)

# Fowlkes-Mallows Index
fmi = fowlkes_mallows_score(true_labels, predicted_labels)

# Adjusted Rand Index
ari = adjusted_rand_score(true_labels, predicted_labels)

# Normalized Mutual Information
nmi = normalized_mutual_info_score(true_labels, predicted_labels)

# Homogeneity, Completeness, V-Measure
homogeneity, completeness, v_measure = homogeneity_completeness_v_measure(true_labels, predicted_labels)

# Purity is not directly available in sklearn, but can be calculated manually

# Calculate purity
def purity_score(y_true, y_pred):
    # Compute contingency matrix
    matrix = contingency_matrix(y_true, y_pred)
    return np.sum(np.amax(matrix, axis=0)) / np.sum(matrix)

# Assuming umap_df contains your data, with 'tumor_type' as true labels and 'cluster' as predicted labels
true_labels = umap_df['tumor_type']
predicted_labels = umap_df['cluster']

purity = purity_score(true_labels, predicted_labels)
#print(f'Purity: {purity}')


#add each metric as a row to the metrics df
metrics_df = pd.concat([metrics_df, pd.DataFrame({'silhouette': [silhouette], 'fmi': [fmi], 'ari': [ari], 'nmi': [nmi], 'homogeneity': [homogeneity], 'completeness': [completeness], 'v_measure': [v_measure], 'purity': [purity]}).T])
metrics_df

In [ ]:
#create a table of all the metrics as well as weighted avg entropy and mi score


subset data with umap

In [ ]:

# Compute the correlation matrix
correlation_matrix = umap_df.corr()
correlation_matrix

In [ ]:
#define grid subset: umap 1 min, max; umap 2 min, max:
box_select = {'UMAP 1': [0, 12], 'UMAP 2': [5, 9]}
# select from umap df:
umap_subset = umap_df.loc[(umap_df['UMAP 1'] > box_select['UMAP 1'][0]) & (umap_df['UMAP 1'] <= box_select['UMAP 1'][1]) & (umap_df['UMAP 2'] > box_select['UMAP 2'][0]) & (umap_df['UMAP 2'] <= box_select['UMAP 2'][1]), :]
#select the opposite as well
umap_subset_opp = umap_df.loc[(umap_df['UMAP 1'] <= box_select['UMAP 1'][0]) | (umap_df['UMAP 1'] > box_select['UMAP 1'][1]) | (umap_df['UMAP 2'] <= box_select['UMAP 2'][0]) | (umap_df['UMAP 2'] > box_select['UMAP 2'][1]), :]

In [ ]:
#print data type for each collumn
for col in umap_subset.columns:
    print(col, umap_subset[col].dtype)


In [ ]:
from scipy.stats import ttest_ind
#create a table of the mean of the numerical cols in each df as well as the result of a 
# two tailed t test

# Identify numerical columns (excluding 'UMAP 1' and 'UMAP 2' as they were used for subsetting)
numerical_cols = umap_df.select_dtypes(include='number').columns.drop(['UMAP 1', 'UMAP 2'])

# Initialize a DataFrame to store the results
results_df = pd.DataFrame(columns=['Column', 'Mean Subset', 'Mean Opposite Subset', 'T-test P-value'])

# Loop through each numerical column to compute means and perform t-tests
for col in numerical_cols:
    mean_subset = umap_subset[col].mean()
    mean_subset_opp = umap_subset_opp[col].mean()

    # Perform two-tailed t-test
    t_test_result = ttest_ind(umap_subset[col], umap_subset_opp[col], nan_policy='omit')

    # Append results to the DataFrame
    results_df = results_df.append({
        'Column': col,
        'Mean Subset': mean_subset,
        'Mean Opposite Subset': mean_subset_opp,
        'T-test P-value': t_test_result.pvalue
    }, ignore_index=True)

# Display the results
results_df

In [ ]:
# Identify categorical columns
categorical_cols = umap_df.select_dtypes(include='category').columns
categorical_cols

In [ ]:
from scipy.stats import chi2_contingency
# First, create a DataFrame without the columns you want to exclude
reduced_df = umap_df.drop(['UMAP 1', 'UMAP 2', 'cell_type', 'Sex'], axis=1)

# Now, select only the categorical columns from this reduced DataFrame
categorical_cols = reduced_df.select_dtypes(include='category').columns

# Initialize a DataFrame to store the results
results_df = pd.DataFrame(columns=['Column', 'Counts in Subset', 'Counts in Opposite Subset', 'Chi-Squared P-value'])

# Loop through each categorical column to count levels and perform chi-squared tests
for col in categorical_cols:
    # Count the levels in each subset
    counts_subset = umap_subset[col].value_counts()
    counts_subset_opp = umap_subset_opp[col].value_counts()

    # Create a contingency table
    contingency_table = pd.DataFrame({
        'Subset': counts_subset,
        'Opposite Subset': counts_subset_opp
    }).fillna(0)

    # Perform chi-squared test
    chi2, p, _, _ = chi2_contingency(contingency_table)

    # Append results to the DataFrame
    results_df = results_df.append({
        'Column': col,
        'Counts in Subset': dict(counts_subset),
        'Counts in Opposite Subset': dict(counts_subset_opp),
        'Chi-Squared P-value': p
    }, ignore_index=True)

# Display the results
results_df

In [ ]:
from scipy.stats import hypergeom


# Extracting the relevant data from results_df
subtype_data = results_df.loc[results_df['Column'] == 'Subtype'].iloc[0]
counts_subset = subtype_data['Counts in Subset']
counts_subset_opp = subtype_data['Counts in Opposite Subset']
chi_squared_p_value = subtype_data['Chi-Squared P-value']

# Getting the union of keys from both count dictionaries
all_subtypes = set(counts_subset.keys()).union(set(counts_subset_opp.keys()))

# Creating a new DataFrame
subtype_counts_df = pd.DataFrame(index=all_subtypes, columns=['Subset', 'Opposite Subset'])

# Populating the DataFrame
for subtype in all_subtypes:
    subtype_counts_df.loc[subtype, 'Subset'] = counts_subset.get(subtype, 0)
    subtype_counts_df.loc[subtype, 'Opposite Subset'] = counts_subset_opp.get(subtype, 0)

# Adding the chi-squared p-value as a new column
subtype_counts_df['Chi-Squared P-value'] = chi_squared_p_value

# Total number of samples in each subset
n_subset = umap_subset.shape[0]
n_subset_opp = umap_subset_opp.shape[0]
total_samples = n_subset + n_subset_opp

# Add a new column for the probability calculation
subtype_counts_df['Random Selection Probability'] = 0

for subtype in all_subtypes:
    # Total number of samples of this subtype in the entire dataset
    total_subtype_samples = counts_subset.get(subtype, 0) + counts_subset_opp.get(subtype, 0)

    # Number of samples of this subtype in the umap_subset
    subtype_samples_in_subset = counts_subset.get(subtype, 0)

    # Calculate the probability using the hypergeometric distribution
    p_value = hypergeom(total_samples, total_subtype_samples, n_subset).pmf(subtype_samples_in_subset)
    subtype_counts_df.loc[subtype, 'Random Selection Probability'] = p_value

# Display the new DataFrame
subtype_counts_df


In [ ]:
cell_embbed.obs.head()

In [ ]:

# Set style for a light background
plt.style.use('seaborn-v0_8-pastel')  # or 'classic'

#create plot
plt.figure(figsize=(12, 10))

# Use the 'oncosig_label' values for coloring the points
# 'cmap' can be adjusted to your preferred color map

# Manually set colors based on 'oncosig_label_ERBB2'
colors = ['darkblue' if label == 0 else 'red' for label in my_adata.obs['oncosig_label_ERBB2']]


scatter = plt.scatter(embedding[:, 0], embedding[:, 1], c=colors, s=8)
plt_title = 'UMAP tcga brca scgpt emb sub genes & onc label'
plt.title(plt_title, fontsize=18)
plt.xlabel('UMAP 1', fontsize=12)
plt.ylabel('UMAP 2', fontsize=12)


# Create a legend
legend_labels = ['Label 0', 'Label 1']
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=legend_labels[i],
                              markerfacecolor=scatter.cmap(scatter.norm(i)), markersize=10)
                   for i in range(2)]
plt.legend(handles=legend_elements, title='Oncosig Labels')

# Save the plot to a file

#plt.savefig(f'plots/{plt_title}.png', dpi=300, bbox_inches='tight')

# Display the plot in the notebook
plt.show()

! compare to raw all genes

In [ ]:
#create plot
raw_all_genes_embedding = reducer.fit_transform(all_genes_adata.X)

In [ ]:


plt.figure(figsize=(12, 10))

# 'cmap' can be adjusted to your preferred color map


# Manually set colors based on 'oncosig_label_ERBB2'
colors = ['darkblue' if label == 0 else 'red' for label in cell_embbed.obs['oncosig_label_ERBB2']]



scatter = plt.scatter(raw_all_genes_embedding[:, 0], raw_all_genes_embedding[:, 1], c=colors, s=7)
plt_title = 'UMAP tcga brca raw all genes emb + onc label'

plt.title(plt_title, fontsize=18)
plt.xlabel('UMAP 1', fontsize=12)
plt.ylabel('UMAP 2', fontsize=12)

# Create a legend
legend_labels = ['Label 0', 'Label 1']
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=legend_labels[i],
                              markerfacecolor=scatter.cmap(scatter.norm(i)), markersize=10)
                   for i in range(2)]
plt.legend(handles=legend_elements, title='Oncosig Labels')
# Save the plot to a file

plt.savefig(f'plots/{plt_title}.png', dpi=300, bbox_inches='tight')
# Display the plot
plt.show()



! compare to umap of maually selected genes

In [ ]:
#create plot
raw_selected_embedding = reducer.fit_transform(my_adata.X)

In [ ]:


plt.figure(figsize=(12, 10))



# Manually set colors based on 'oncosig_label_ERBB2'
colors = ['darkblue' if label == 0 else 'red' for label in cell_embbed.obs['oncosig_label_ERBB2']]



scatter = plt.scatter(raw_selected_embedding[:, 0], raw_selected_embedding[:, 1], c=colors, s=7)
plt_title = 'UMAP tcga brca raw selected genes emb + onc label'

plt.title(plt_title, fontsize=18)
plt.xlabel('UMAP 1', fontsize=12)
plt.ylabel('UMAP 2', fontsize=12)

# Create a legend
legend_labels = ['Label 0', 'Label 1']
legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=legend_labels[i],
                              markerfacecolor=scatter.cmap(scatter.norm(i)), markersize=10)
                   for i in range(2)]
plt.legend(handles=legend_elements, title='Oncosig Labels')


# Save the plot to a file

plt.savefig(f'plots/{plt_title}.png', dpi=300, bbox_inches='tight')

# Display the plot
plt.show()


In [ ]:
plt.style.available

In [ ]:
cell_embbed.write_h5ad('data/brca_scrna_epithelial_scGPT.h5ad')

# check corr between X and obs

In [ ]:
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd

In [ ]:
#drop some of the meta data cols
cell_embbed.obs.columns
cols_drop = [ 'cell_type',
       'Neoplasm Disease Stage American Joint Committee on Cancer Code',
             'MSI MANTIS Score', 'MSIsensor Score', 'Mutation Count',
 'Overall Survival Status',
       'American Joint Committee on Cancer Metastasis Stage Code',
       'American Joint Committee on Cancer Tumor Stage Code',
     'Progression Free Status',
   'Sex', 'Somatic Status', ]
cell_embbed.obs = cell_embbed.obs.drop(cols_drop, axis=1)

In [ ]:
# get data frames and scale


# Convert the X matrix to a DataFrame
values_df = pd.DataFrame(cell_embbed.X, columns=[f'X_{i}' for i in range(cell_embbed.X.shape[1])])

metadata = pd.DataFrame(cell_embbed.obs.copy().drop(['Patient ID'], axis=1))


In [ ]:
#get numerical cols as num_metadata
num_meta_cols = metadata.select_dtypes(include='number').columns

#get cols of values

#get a list of strings: Dim_0, Dim_2, until Dim_511
val_cols = values_df.columns

In [ ]:
#scale
#transform the vals df to log p 1
values_df = np.log1p(values_df)
#scale the vals df 0 to 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
values_df = scaler.fit_transform(values_df)
#scale the numerical values of the metadata
metadata[num_meta_cols] = scaler.fit_transform(metadata[num_meta_cols])


In [ ]:
# generate null MI with bootstrapping
null_mi_distribution = []
#take a number of bootsraps ~ TWICE NUMBER OF COLUMNS

n_bootstraps = 1000

# set threshold of MI
mi_threshold = 0.95
#choose 3 random test runs on which to check the data
test_runs = np.random.randint(0, n_bootstraps, size=3)
test_runs= np.append(test_runs, 0)
test_runs

# Flatten the DataFrame into a 1D NumPy array
flattened_values = values_df.flatten()

In [ ]:
# find null mi dist using boot strap.
for i in range(n_bootstraps):
    # Randomly draw values from the flattened array
    resampled_values = np.random.choice(flattened_values, size=len(values_df))

    # Randomly select a column from metadata
    random_meta_col = np.random.choice(metadata[num_meta_cols].columns)
    selected_metadata = metadata[random_meta_col]

    # Handle NA values - drop rows with NA in either column
    na_mask =  selected_metadata.isna()
    selected_values = resampled_values[~na_mask]
    selected_metadata = selected_metadata[~na_mask]

    #sanity check
    # if i in test_runs:
    #     #print length of bothg cols
    #     print('selected values len:', len(selected_values))
    #     print('selected metadata len:', len(selected_metadata))
    #     print('shapes: val, meta: \n' , selected_values.shape, selected_metadata.shape)
    #     #print the first 5 rows of both cols
    #     print('selected values 1:5:', selected_values[:5])
    #     print('selected metadata 1:5:', selected_metadata[:5])

    # If the metadata column is categorical, encode it
    if selected_metadata.dtype == 'object' or selected_metadata.dtype.name == 'category':
        encoder = LabelEncoder()
        selected_metadata = encoder.fit_transform(selected_metadata)


    # Calculate MI with the resampled data
    mi = mutual_info_score(selected_values, selected_metadata)
    null_mi_distribution.append(mi)

# sort, and print the MI value at threshold percentile. do not rescale!
null_mi_distribution = np.sort(null_mi_distribution)
mi_threshold_val = null_mi_distribution[int(mi_threshold * len(null_mi_distribution))]
print('MI threshold:', mi_threshold_val)


In [ ]:
selected_metadata = metadata['Subtype']

# Handle NA values - drop rows with NA in either column
na_mask =  selected_metadata.isna()

selected_metadata = selected_metadata[~na_mask]
  # If the metadata column is categorical, encode it
if selected_metadata.dtype == 'object' or selected_metadata.dtype.name == 'category':
    encoder = LabelEncoder()
    selected_metadata = encoder.fit_transform(selected_metadata)

In [ ]:

curr_vals = values_df[:,10]
curr_vals = curr_vals[~na_mask]

In [ ]:

# Calculate MI with the resampled data
actual_mi = mutual_info_score(selected_values, selected_metadata)
actual_mi

In [ ]:

#check if any na in val df numpy arr
np.isnan(values_df).any()

In [ ]:
#init a db: val_col, mi, p_val
mi_df = pd.DataFrame(columns=['val_col', 'mi', 'p_val'])

selected_metadata = metadata['Buffa Hypoxia Score']

# Handle NA values - drop rows with NA in either column
na_mask =  selected_metadata.isna()

selected_metadata = selected_metadata[~na_mask]
  # If the metadata column is categorical, encode it
if selected_metadata.dtype == 'object' or selected_metadata.dtype.name == 'category':
    encoder = LabelEncoder()
    selected_metadata = encoder.fit_transform(selected_metadata)

for val_col in range(values_df.shape[1]):
    curr_vals = values_df[:,val_col]
    curr_vals = curr_vals[~na_mask]

    # Calculate MI with the resampled data
    actual_mi = mutual_info_score(curr_vals, selected_metadata)

    #check mi above threshold
    if actual_mi > mi_threshold_val:
        #calc p val:
        print('actual mi:', actual_mi)
        # Calculate the percentile of the actual MI value
        percentile = np.sum(null_mi_distribution <= actual_mi) / len(null_mi_distribution)
        p_value = 1 - percentile
        #add to df
    mi_df = mi_df.append({'val_col': val_col, 'mi': actual_mi, 'p_val': p_value}, ignore_index=True)

In [ ]:
# calc MI between each col in metadata and each col in values
# create a dict whose keys are the meta data cols, and the values are lists of tuples of (col, mi, p-val) of cols whose mi was above the threshold
mi_dict = {}
for meta_col in metadata.columns:
    selected_metadata = metadata[meta_col]

    # Handle NA values - drop rows with NA in either column
    na_mask =  selected_metadata.isna()

    selected_metadata = selected_metadata[~na_mask]
      # If the metadata column is categorical, encode it
    if selected_metadata.dtype == 'object' or selected_metadata.dtype.name == 'category':
        encoder = LabelEncoder()
        selected_metadata = encoder.fit_transform(selected_metadata)

    for val_col in range(values_df.shape[1]):
        curr_vals = values_df[:,val_col]
        curr_vals = curr_vals[~na_mask]

        # Calculate MI with the resampled data
        actual_mi = mutual_info_score(curr_vals, selected_metadata)

        #check mi above threshold
        if actual_mi > mi_threshold_val:
            #calc p val:
            print('actual mi:', actual_mi)
            # Calculate the percentile of the actual MI value
            percentile = np.sum(null_mi_distribution <= actual_mi) / len(null_mi_distribution)
            p_value = 1 - percentile
            #add to dict
            if meta_col in mi_dict.keys():
                mi_dict[meta_col].append((val_col, actual_mi, p_value))
            else:
                mi_dict[meta_col] = [(val_col, actual_mi, p_value)]


In [ ]:
# find the cols with highest MI for each col in metadata





In [ ]:
# get corrs for scaled data

# Identify numerical columns (excluding 'UMAP 1' and 'UMAP 2' as they were used for subsetting)
numerical_cols = cell_embbed.obs.select_dtypes(include='number')
# Initialize a DataFrame to store the highest correlation for each numerical column
highest_correlations = pd.DataFrame(columns=['obs_column', 'X_column', 'correlation'])


# Loop through each column in the obs matrix
for obs_col in metadata.columns:
    #print('obs col 1[1]:', cell_embbed.obs[obs_col][0])
    # Initialize a variable to store the highest correlation for this obs column
    highest_corr = {'obs_column': obs_col, 'X_column': None, 'correlation': 0}



      # If the metadata column is categorical, encode it
    if selected_metadata.dtype == 'object' or selected_metadata.dtype.name == 'category':
        #move to next col
        continue
    # Loop through each column in the X matrix
    for val_col in range(values_df.shape[1]):
        curr_vals = values_df[:,val_col]
        #convert curr vals to pd series
        curr_vals = pd.Series(curr_vals)
        corr = cell_embbed.obs[obs_col].corr(curr_vals)
        print('corr:', corr)
        # Check if this is the highest correlation so far for this obs column
        if abs(corr) > abs(highest_corr['correlation']):
            highest_corr['X_column'] = val_col
            highest_corr['correlation'] = corr

    print(highest_corr)
    # Append the highest correlation for this obs column to the DataFrame
    highest_correlations = highest_correlations.append(highest_corr, ignore_index=True)

# Display the results
highest_correlations


In [ ]:


# Identify numerical columns (excluding 'UMAP 1' and 'UMAP 2' as they were used for subsetting)
numerical_cols = cell_embbed.obs.select_dtypes(include='number')
# Initialize a DataFrame to store the highest correlation for each numerical column
highest_correlations = pd.DataFrame(columns=['obs_column', 'X_column', 'correlation'])


# Loop through each column in the obs matrix
for obs_col in metadata.columns:
    #print('obs col 1[1]:', cell_embbed.obs[obs_col][0])
    # Initialize a variable to store the highest correlation for this obs column
    highest_corr = {'obs_column': obs_col, 'X_column': None, 'correlation': 0}

    # Loop through each column in the X matrix
    for x_col in X_df.columns:
        #print('x col 1[1]:', X_df[x_col][0])
        # Compute the correlation
        corr = cell_embbed.obs[obs_col].corr(X_df[x_col])
        #print('corr:', corr)
        # Check if this is the highest correlation so far for this obs column
        if abs(corr) > abs(highest_corr['correlation']):
            highest_corr['X_column'] = x_col
            highest_corr['correlation'] = corr

    print(highest_corr)
    # Append the highest correlation for this obs column to the DataFrame
    highest_correlations = highest_correlations.append(highest_corr, ignore_index=True)

# Display the results
highest_correlations


In [ ]:
# and now with MI:


# Initialize a DataFrame to store the highest mutual information for each numerical column
highest_mutual_info = pd.DataFrame(columns=['obs_column', 'X_column', 'mutual_info'])

# Loop through each column in the obs matrix
for obs_col in numerical_cols.columns:
    # Initialize a variable to store the highest mutual information for this obs column
    highest_mi = {'obs_column': obs_col, 'X_column': None, 'mutual_info': 0}

    # Select and process the obs column, drop NA values
    curr_obs = cell_embbed.obs[obs_col].dropna()
    print('curr obs:', obs_col, len(curr_obs))

    # Loop through each column in the X matrix
    for x_col in X_df.columns:
        # Compute the mutual information
         # Select the x_col values, aligning with curr_obs by index
        x_col_vals = X_df.loc[curr_obs.index, x_col]

        mi = mutual_info_score(curr_obs, x_col_vals)


        # Check if this is the highest mutual information so far for this obs column
        if mi > highest_mi['mutual_info']:
            highest_mi['X_column'] = x_col
            highest_mi['mutual_info'] = mi

    # Append the highest mutual information for this obs column to the DataFrame
    highest_mutual_info = highest_mutual_info.append(highest_mi, ignore_index=True)

# Display the results
print(highest_mutual_info)

# export notebook as script

In [ ]:
nb_path = Path(base_dir / 'scgpt/j_scgpt_utils/cell_emb_plot.ipynb')
output_path = Path(Path(base_dir / 'scgpt/j_scgpt_utils/cell_emb_plot.py'))
short_utils.export_marked_cells(nb_path, output_path)